In [1]:
import os
import sys
from keras.models import Model
from keras.layers import concatenate
from keras.utils.data_utils import get_file
from keras.optimizers import SGD
from data_generator.config import mpii_sp_dataconf
from data_generator.mpii import MpiiSinglePerson
from data_generator.loader import BatchLoader
from models.losses import pose_regression_loss
from models import reception
from models.mpii_tools import eval_singleperson_pckh
from models.ntu_tools import NtuEvalCallback
from keras.callbacks import LearningRateScheduler
from models.callbacks import SaveModel
from data_generator.io_operate import *
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
from data_generator.pose import *
from data_generator.ntu import Ntu
from data_generator.config import ntu_ar_dataconf
from models import action

Using TensorFlow backend.


In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))
num_frames = 12
num_blocks = 4
batch_size = 1
depth_maps = 8
num_joints = 20
num_actions = 49
pose_dim = 3
input_shape = ntu_ar_dataconf.input_shape
dataset_path = './data/ntu'
TEST_MODE = 0
TRAIN_MODE = 1
VALID_MODE = 2
logdir = './ntu_action'
pose_model_weights = './ntu_pose/weights_ntu_pos_120.h5'

In [3]:
ntu = Ntu(dataset_path, ntu_ar_dataconf, poselayout=pa20j3d,
        topology='sequences', use_gt_bbox=True, clip_size=num_frames)

ntu_tr = BatchLoader(ntu, ['frame'],['ntuaction'], TRAIN_MODE,\
                     num_predictions=num_blocks,batch_size=batch_size)
ntu_val = ntu_te = BatchLoader(ntu, ['frame'], ['ntuaction'], VALID_MODE,\
        batch_size=1, shuffle=False)

Max/Min number of frames: 158/54
Number of videos: [0, 5, 5]


In [4]:
model_pe = reception.build(input_shape, num_joints, dim=pose_dim,\
                           num_blocks=num_blocks, depth_maps=depth_maps, ksize=(5, 5))

In [5]:
model_pe.load_weights(pose_model_weights)

In [6]:
model = action.build_merge_model(model_pe, num_actions, input_shape,
        num_frames, num_joints, num_blocks, pose_dim=pose_dim,
        num_context_per_joint=0, pose_net_version='v2')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
Stem (Model)                    (None, 32, 32, 576)  1039488     input_1[0][0]                    
__________________________________________________________________________________________________
rBlock1 (Model)                 (None, 32, 32, 576)  1312128     Stem[1][0]                       
__________________________________________________________________________________________________
SepConv1 (Model)                (None, 32, 32, 576)  347904      rBlock1[1][0]                    
__________________________________________________________________________________________________
RegMap1 (M

In [8]:
eval_callback = NtuEvalCallback(ntu_val,eval_model=model,num_actions = num_actions,logdir=logdir)

In [9]:
optimizer = SGD(lr=0.0002, momentum=0.98)
model.compile(loss='categorical_crossentropy', optimizer = optimizer)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           (None, 12, 256, 256, 0                                            
__________________________________________________________________________________________________
td_Stem (TimeDistributed)       (None, 12, 32, 32, 5 1039488     input_21[0][0]                   
__________________________________________________________________________________________________
td_rBlock1 (TimeDistributed)    (None, 12, 32, 32, 5 1312128     td_Stem[0][0]                    
__________________________________________________________________________________________________
td_Model1 (TimeDistributed)     (None, 12, 32, 32, 1 5978304     td_rBlock1[0][0]                 
__________________________________________________________________________________________________
lambda_27 

In [10]:
def lr_scheduler(epoch, lr):

    if epoch in [80, 100]:
        newlr = 0.2*lr
        printcn(WARNING, 'lr_scheduler: lr %g -> %g @ %d' % (lr, newlr, epoch))
    elif epoch == 120:
        newlr = 0.1*lr
        printcn(WARNING, 'lr_scheduler: lr %g -> %g @ %d' % (lr, newlr, epoch))
    else:
        newlr = lr
        printcn(OKBLUE, 'lr_scheduler: lr %g @ %d' % (newlr, epoch))

    return newlr
weights_file = 'weights_ntu_action_{epoch:03d}.h5'
callbacks = []
callbacks.append(SaveModel(os.path.join(logdir,weights_file)))
callbacks.append(LearningRateScheduler(lr_scheduler))
callbacks.append(eval_callback)

In [11]:
model.fit_generator(ntu_tr,
        epochs=125,
        callbacks=callbacks,
        workers=4,
        initial_epoch=0)

Epoch 1/125
lr_scheduler: lr 0.0002 @ 0
5/5 [==============================] - 15s 3s/step - loss: 23.7037 - p_v1_loss: 7.1239 - p_v2_loss: 5.7844 - p_v3_loss: 5.8499 - p_v4_loss: 4.9455
Saving model @epoch=00001 to ./ntu_action/weights_ntu_action_001.h5
pred 00001/00005
pred 00002/00005
pred 00003/00005
pred 00004/00005
pred 00005/00005
NTU, single-clip, GT bbox, action acc.%: 0.0  0.0  0.0  20.0 drink water: 0.0
eat meal/snack: 0.0
brushing teeth: 1.0
brushing hair: 0.0
drop: 0.0

5 samples in 2.5 sec: 2.0 clips per sec
Best score is 20.0 in blocks 4 at epoch 1
Epoch 2/125
lr_scheduler: lr 0.0002 @ 1
5/5 [==============================] - 2s 346ms/step - loss: 14.1313 - p_v1_loss: 4.0031 - p_v2_loss: 3.2718 - p_v3_loss: 3.8390 - p_v4_loss: 3.0175
Saving model @epoch=00002 to ./ntu_action/weights_ntu_action_002.h5
pred 00001/00005
pred 00002/00005
pred 00003/00005
pred 00004/00005
pred 00005/00005
NTU, single-clip, GT bbox, action acc.%: 20.0  40.0  0.0  40.0 drink water: 0.0
eat meal

KeyboardInterrupt: 

In [ ]:
import json
import numpy as np

In [ ]:
path = './ntu_pose/ntu_pose_val.json'
max_score = np.inf
max_block = 0
max_epoch = 0
with open(path) as f:
    record = json.load(f)
    for key,value in record.items():
        score = min(value[0])
        if score <= max_score:
            max_score = score
            max_block = np.argmin(value[0])+1
            max_epoch = key

In [ ]:
max_epoch

In [ ]:
max_block

In [ ]:
max_score